In [28]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [6]:
url = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=1&ref_=adv_nxt"
r = rq.get(url)
sopa = bs(r.content, "html.parser")

In [9]:
lista_pelis = sopa.find_all('div', class_='lister-item mode-advanced')
lista_pelis

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt0111161"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt0111161/"> <img alt="Sueños de Libertad" class="loadlate" data-tconst="tt0111161" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt0111161/">Sueños de Libertad</a>
 <span class="lister-item-year text-muted unbold">(1994)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">18</span>
 <span class="ghost">|</span>
 <span class="runtime">142 min</span>
 <span class="ghost">|</span>
 <span class="genr

In [14]:
peli1 = lista_pelis[0]
peli1_contenido = peli1.find('div', class_='lister-item-content')
peli1_contenido

<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt0111161/">Sueños de Libertad</a>
<span class="lister-item-year text-muted unbold">(1994)</span>
</h3>
<p class="text-muted">
<span class="certificate">18</span>
<span class="ghost">|</span>
<span class="runtime">142 min</span>
<span class="ghost">|</span>
<span class="genre">
Drama            </span>
</p>
<div class="ratings-bar">
<div class="inline-block ratings-imdb-rating" data-value="9.3" name="ir">
<span class="global-sprite rating-star imdb-rating"></span>
<strong>9.3</strong>
</div>
<div class="inline-block ratings-user-rating">
<span class="userRatingValue" data-tconst="tt0111161" id="urv_tt0111161">
<span class="global-sprite rating-star no-rating"></span>
<span class="rate" data-no-rating="Rate this" data-value="0" name="ur">Rate this</span>
</span>
<div class="starBarWidget" id="sb_tt0111161">
<div class="rating rating-list" data-c

In [110]:
titulos = []
anos = []
edades = []
duraciones = []
generos = []
ratings = []
directores = []
votos = []
ganancias = []

for data in lista_pelis:

    contenido = data.find('div', class_='lister-item-content')
    cabecera = contenido.find('h3')
    titulo = cabecera.find('a')
    ano = cabecera.find('span', class_='lister-item-year text-muted unbold')
    ano = ano.text.replace('(', '')
    ano = ano.replace(')', '')
    titulos.append(titulo.text)
    anos.append(ano)
    
    cuerpo = contenido.find('p')
    edad = cuerpo.find('span', class_= 'certificate')
    tiempo = cuerpo.find('span', class_ = 'runtime')
    genero = cuerpo.find('span', class_= 'genre')
    if edad:
        edades.append(edad.text)
    else:
        edades.append(np.nan)
    duraciones.append(tiempo.text)
    generos.append(genero.text.strip())

    cuerpo2 = contenido.find('div', class_= 'ratings-bar')
    rating = cuerpo2.find('div', class_='inline-block ratings-imdb-rating')
    ratings.append(rating.text.strip())

    pie = contenido.find('p', class_='')
    pie = pie.text.strip().split('|')
    director = pie[0]
    director = director.split('\n')
    directores.append(director[1])

    pie2 = contenido.find('p', class_='sort-num_votes-visible')
    pie2 = pie2.text.strip().split('|')
    voto = pie2[0].split(':')
    voto = voto[1].strip()
    voto = voto.replace(',', '')
    if len(pie2) == 3:
        precio = pie2[1].split(':')
        precio = precio[1].strip()
    else:
        precio = np.nan
    votos.append(voto)
    ganancias.append(precio)

In [111]:
df_pelis = pd.DataFrame()
df_pelis['Titulo'] = titulos
df_pelis['Ano'] = anos
df_pelis['Edad_min'] = edades
df_pelis['duraciones'] = duraciones
df_pelis['Genero'] = generos
df_pelis['Rating'] = ratings
df_pelis['Director'] = directores
df_pelis['Votos'] = votos
df_pelis['Ganancias'] = ganancias
df_pelis

,Titulo,Ano,Edad_min,duraciones,Genero,Rating,Director,Votos,Ganancias
0,Sueños de Libertad,1994,18,142 min,Drama,9.3,Frank Darabont,2663568,$28.34M
1,El padrino,1972,NaN,175 min,"Crime, Drama",9.2,Francis Ford Coppola,1845841,$134.97M
2,Kantara,2022,NaN,148 min,"Action, Adventure, Drama",9.0,Rishab Shetty,68144,NaN
3,The Dark Knight,2008,PG-13,152 min,"Action, Crime, Drama",9.0,Christopher Nolan,2636550,$534.86M
4,The Lord of the Rings: The Return of the King,2003,PG-13,201 min,"Action, Adventure, Drama",9.0,Peter Jackson,1836397,$377.85M
...,...,...,...,...,...,...,...,...,...
95,Paths of Glory,1957,NaN,88 min,"Drama, War",8.4,Stanley Kubrick,197764,NaN
96,El ocaso de una vida,1950,NaN,110 min,"Drama, Film-Noir",8.4,Billy Wilder,222287,NaN
97,El gran dictador,1940,NaN,125 min,"Comedy, Drama, War",8.4,Charles Chaplin,224254,$0.29M
98,Chhichhore,2019,NaN,143 min,"Comedy, Drama, Romance",8.3,Nitesh Tiwari,54741,NaN


In [113]:
df_pelis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Titulo      100 non-null    object
 1   Ano         100 non-null    object
 2   Edad_min    70 non-null     object
 3   duraciones  100 non-null    object
 4   Genero      100 non-null    object
 5   Rating      100 non-null    object
 6   Director    100 non-null    object
 7   Votos       100 non-null    object
 8   Ganancias   75 non-null     object
dtypes: object(9)
memory usage: 7.2+ KB
